Function calling is a powerful technique that allows you to extend the capabilities of a language model like Claude by integrating external functions or APIs. It enables the model to perform tasks or access information that it wouldn't be able to do on its own.



1. **Function Definition**: First, you need to define the external function that you want Claude to call. This function can perform any task, such as string manipulation, mathematical calculations, API queries, or database operations. In the string reversal example, we defined the `reverse_string` function:

```python
def reverse_string(string):
    return string[::-1]
```

2. **Tool Description**: Next, you create a description of the function (referred to as a "tool") that Claude can understand. This description includes the tool's name, a brief explanation of what it does, and the parameters it accepts. You use the `construct_format_tool_for_claude_prompt` function to format this information:

```python
tool_name = "reverse_string"
tool_description = "Reverses the provided string."

parameters = [
    {
        "name": "string",
        "type": "str",
        "description": "The string to be reversed."
    }
]

tool = construct_format_tool_for_claude_prompt(tool_name, tool_description, parameters)
```

3. **System Prompt**: The system prompt is a set of instructions that guide Claude on how to use the available tools. It's created using the `construct_tool_use_system_prompt` function, which takes the tool as input:

```python
system_prompt = construct_tool_use_system_prompt([tool])
```

4. **User Message**: The user message is the input from the user that Claude will respond to. In the example, it's a request to reverse a specific string:

```python
reverse_message = {
    "role": "user",
    "content": "Reverse the string 'Hello, World!'"
}
```

5. **Partial Response**: Claude processes the user message and the system prompt to generate a partial response. This response includes a function call in a specific format. The `client.messages.create` function is used to send the message to Claude and retrieve the partial response:

```python
function_calling_message = client.messages.create(
    model="claude-3-opus-20240229",
    max_tokens=1024,
    messages=[reverse_message],
    system=system_prompt,
    stop_sequences=["\n\nHuman:", "\n\nAssistant", "</function_calls>"]
).content[0].text
```

6. **Parameter Extraction**: The parameters for the function call are extracted from the partial response using the `extract_between_tags` function. This function uses regular expressions to find the values between specific XML-like tags:

```python
string = extract_between_tags("string", function_calling_message)[0]
```

7. **Function Execution**: The extracted parameters are passed to the actual function (`reverse_string` in this case), and the function is executed:

```python
reversed_string = reverse_string(string)
```

8. **Result Formatting**: The result of the function execution is formatted into a structure that Claude can understand using the `construct_successful_function_run_injection_prompt` function:

```python
formatted_results = [{
    'tool_name': 'reverse_string',
    'tool_result': reversed_string
}]
function_results = construct_successful_function_run_injection_prompt(formatted_results)
```

9. **Final Response**: The formatted results are appended to the partial response, and this combined message is sent back to Claude to generate the final response:

```python
partial_assistant_message = function_calling_message + "</function_calls>" + function_results

final_message = client.messages.create(
    model="claude-3-opus-20240229",
    max_tokens=1024,
    messages=[
        reverse_message,
        {
            "role": "assistant",
            "content": partial_assistant_message
        }
    ],
    system=system_prompt
).content[0].text
```

In the final response, Claude incorporates the result of the function call into its output, allowing it to provide a response that it wouldn't have been able to generate on its own.

Function calling opens up a wide range of possibilities for extending the capabilities of language models like Claude. By defining custom functions and providing clear descriptions of how to use them, you can enable Claude to perform complex tasks, access real-time data, and interact with external systems.

In [1]:
# !pip install anthropic --upgrade

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()
api_key = os.getenv("ANTHROPIC_API_KEY")

In [2]:
import anthropic
import re
import json

client = anthropic.Anthropic(
    # defaults to os.environ.get("ANTHROPIC_API_KEY")
    api_key=api_key,
)


In [3]:
def construct_format_parameters_prompt(parameters):
    constructed_prompt = "\n".join(
        f"<parameter>\n<name>{parameter['name']}</name>\n<type>{parameter['type']}</type>\n<description>{parameter['description']}</description>\n</parameter>"
        for parameter in parameters
    )
    return constructed_prompt

In [4]:
def construct_format_tool_for_claude_prompt(name, description, parameters):
    constructed_prompt = (
        "<tool_description>\n"
        f"<tool_name>{name}</tool_name>\n"
        "<description>\n"
        f"{description}\n"
        "</description>\n"
        "<parameters>\n"
        f"{construct_format_parameters_prompt(parameters)}\n"
        "</parameters>\n"
        "</tool_description>"
    )
    return constructed_prompt

In [5]:
def construct_tool_use_system_prompt(tools):
    tool_use_system_prompt = (
        "In this environment you have access to a set of tools you can use to answer the user's question.\n"
        "\n"
        "You may call them like this:\n"
        "<function_calls>\n"
        "<invoke>\n"
        "<tool_name>$TOOL_NAME</tool_name>\n"
        "<parameters>\n"
        "<$PARAMETER_NAME>$PARAMETER_VALUE</$PARAMETER_NAME>\n"
        "...\n"
        "</parameters>\n"
        "</invoke>\n"
        "</function_calls>\n"
        "\n"
        "Here are the tools available:\n"
        "<tools>\n"
        + '\n'.join([tool for tool in tools]) +
        "\n</tools>"
    )
    return tool_use_system_prompt

In [6]:
def extract_between_tags(tag: str, string: str, strip: bool = False) -> list[str]:
    ext_list = re.findall(f"<{tag}>(.+?)</{tag}>", string, re.DOTALL)
    if strip:
        ext_list = [e.strip() for e in ext_list]
    return ext_list

In [26]:
message = client.messages.create(
    model="claude-3-opus-20240229",
    max_tokens=1024,
    messages=[
        {
            "role": "user", 
            "content": "Reverse the string encyclopediachatgpt"
        },

    ]
).content[0].text
print(message)

tpgtahcaidepcolcycne


In [28]:
"tpgtahcaidepcolcycne" == "encyclopediachatgpt"[::-1]

False

Most large language models (LLMs) struggle with string reversal tasks because they are primarily trained on natural language data and lack explicit programming knowledge. 

In [9]:
def reverse_string(string):
    return string[::-1]

In [10]:
tool_name = "reverse_string"
tool_description = "Reverses the provided string."

parameters = [
    {
        "name": "string",
        "type": "str",
        "description": "The string to be reversed."
    }
]

tool = construct_format_tool_for_claude_prompt(tool_name, tool_description, parameters)
system_prompt = construct_tool_use_system_prompt([tool])

In [11]:
print(tool)

<tool_description>
<tool_name>reverse_string</tool_name>
<description>
Reverses the provided string.
</description>
<parameters>
<parameter>
<name>string</name>
<type>str</type>
<description>The string to be reversed.</description>
</parameter>
</parameters>
</tool_description>


In [12]:
print(system_prompt)

In this environment you have access to a set of tools you can use to answer the user's question.

You may call them like this:
<function_calls>
<invoke>
<tool_name>$TOOL_NAME</tool_name>
<parameters>
<$PARAMETER_NAME>$PARAMETER_VALUE</$PARAMETER_NAME>
...
</parameters>
</invoke>
</function_calls>

Here are the tools available:
<tools>
<tool_description>
<tool_name>reverse_string</tool_name>
<description>
Reverses the provided string.
</description>
<parameters>
<parameter>
<name>string</name>
<type>str</type>
<description>The string to be reversed.</description>
</parameter>
</parameters>
</tool_description>
</tools>


In [15]:
reverse_message = {
    "role": "user",
    "content": "Reverse the string encyclopedia"
}

In [16]:
function_calling_message = client.messages.create(
    model="claude-3-opus-20240229",
    max_tokens=1024,
    messages=[reverse_message],
    system=system_prompt,
    stop_sequences=["\n\nHuman:", "\n\nAssistant", "</function_calls>"]
).content[0].text

print(function_calling_message)

Okay, here is how to reverse the string "encyclopedia":

<function_calls>
<invoke>
<tool_name>reverse_string</tool_name>
<parameters>
<string>encyclopedia</string>
</parameters>
</invoke>



In [22]:
string = extract_between_tags("string", function_calling_message)[0]
reversed_string = reverse_string(string)

In [23]:
def construct_successful_function_run_injection_prompt(invoke_results):
    constructed_prompt = (
        "<function_results>\n"
        + '\n'.join(
            f"<result>\n<tool_name>{res['tool_name']}</tool_name>\n<stdout>\n{res['tool_result']}\n</stdout>\n</result>" 
            for res in invoke_results
        ) + "\n</function_results>"
    )
    
    return constructed_prompt

In [24]:
formatted_results = [{
    'tool_name': 'reverse_string',
    'tool_result': reversed_string
}]
function_results = construct_successful_function_run_injection_prompt(formatted_results)

In [26]:
print(function_results)

<function_results>
<result>
<tool_name>reverse_string</tool_name>
<stdout>
aidepolcoycne
</stdout>
</result>
</function_results>


In [27]:
partial_assistant_message = function_calling_message + "</function_calls>" + function_results

final_message = client.messages.create(
    model="claude-3-opus-20240229",
    max_tokens=1024,
    messages=[
        reverse_message,
        {
            "role": "assistant",
            "content": partial_assistant_message
        }
    ],
    system=system_prompt
).content[0].text

print(partial_assistant_message + final_message)

Here is the result of reversing the string "encyoclopedia":
<function_calls>
<invoke>
<tool_name>reverse_string</tool_name>
<parameters>
<string>encyoclopedia</string>
</parameters>
</invoke>
</function_calls><function_results>
<result>
<tool_name>reverse_string</tool_name>
<stdout>
aidepolcoycne
</stdout>
</result>
</function_results>

So the reversed string is "aidepolcoycne".


In [28]:
print(final_message)



So the reversed string is "aidepolcoycne".


# Another Approach

# Tools 

## Anthropic Tools Installation

Function calling is a technique that allows you to extend the capabilities of a language model like Claude by integrating external functions or APIs. It enables the model to perform tasks or access information that it wouldn't be able to do on its own.

The process involves defining a set of tools (functions) that Claude can use, and then providing a structured prompt format that guides Claude on how to use these tools. The prompt format includes a description of each tool, its parameters, and how to call it.

When Claude receives a user's message, it analyzes the message and determines if it needs to use any of the available tools to provide a response. If it does, it generates a special message (called a "tool_inputs" message) that specifies which tool it wants to use and what arguments it wants to pass to that tool.

At this point, there are two modes of operation: automatic and manual.

1. Automatic Mode (execution_mode='automatic'):
   - In automatic mode, the tool use process is handled automatically by the ToolUser class.
   - When Claude generates a "tool_inputs" message, the ToolUser automatically extracts the specified tool and its arguments, calls the corresponding function, and generates a "tool_outputs" message with the result.
   - This process continues until Claude generates a response that doesn't require any tool use (i.e., a regular "assistant" message).
   - Automatic mode is simpler to use but provides less control over the tool use process.

2. Manual Mode (execution_mode='manual'):
   - In manual mode, you have more control over the tool use process.
   - When Claude generates a "tool_inputs" message, the ToolUser stops and returns this message to you.
   - You are then responsible for extracting the specified tool and its arguments, calling the corresponding function, and generating a "tool_outputs" message with the result.
   - You pass this "tool_outputs" message back to the ToolUser, which sends it to Claude.
   - This process continues, with you manually handling each "tool_inputs" message, until Claude generates a regular "assistant" message.
   - Manual mode requires more code to handle the back-and-forth between Claude and the tools, but it allows you to add your own validation logic, custom error handling, and more.

Regardless of the mode, the key idea is that Claude can ask to use external tools when it needs them, and the ToolUser facilitates the communication between Claude and these tools.

This function calling technique allows you to leverage the language understanding and generation capabilities of Claude, while augmenting it with additional capabilities provided by external functions. It's a powerful way to create more capable and versatile AI systems.

Some common use cases for function calling include:
- Performing complex calculations or data transformations
- Retrieving real-time data from APIs
- Accessing databases or knowledge bases
- Executing actions in external systems



In [29]:
# !git clone https://github.com/anthropics/anthropic-tools.git
# !cd anthropic-tools && pip install -r requirements.txt
# !cd anthropic-tools && cp -r tool_use_package ../

# Tools with Execution Mode: Automatic

In [30]:
import requests

from tool_use_package.tools.base_tool import BaseTool
from tool_use_package.tool_user import ToolUser

In [31]:
class StringReversalTool(BaseTool):
    """Reverses a given string."""

    def use_tool(self, string: str):
        """Reverses the characters in the given string."""
        reversed_string = string[::-1]
        return {"reversed_string": reversed_string}


In [32]:
tool_name = "reverse_string"
tool_description = """The reverse_string tool will return the reversed version of the given string."""
tool_parameters = [
    {"name": "string", "type": "str", "description": "The string to be reversed."}
]

In [33]:
string_reversal_tool = StringReversalTool(tool_name, tool_description, tool_parameters)


In [35]:
# Pass the tool instance into the ToolUser
tool_user = ToolUser([string_reversal_tool])


In [36]:
# Call the tool_user with a prompt to get a version of Claude that can use your tools!
messages = [{"role": "user", "content": "Reverse the string 'encyclopeida'"}]
print(tool_user.use_tools(messages, execution_mode='automatic'))



So the reversed string is 'adiepolcycne'.


# Tools with Execution Mode: Manual

In [37]:
# Define the function to handle manual Claude responses
def handle_manual_claude_res(messages, claude_res, tool_user):
    """
    - messages does not include claude_res
    - tool_user should be the ToolUser instance you have been using for previous messages
    """
    # Append Claude's response to messages.
    messages.append(claude_res)
    
    if claude_res['role'] == "assistant":
        # If the message is not trying to use a tool we should not automatically respond to Claude, and instead we should ask the user for input.
        return {"next_action": "user_input", "messages": messages}
    elif claude_res['role'] == "tool_inputs":
        # If the message is trying to use a tool we should parse the tool and arguments, use the tool, create the tool_outputs message with the results, and append that message to messages.
        tool_outputs = []
        for tool_input in claude_res['tool_inputs']:
            tool = next((t for t in tool_user.tools if t.name == tool_input['tool_name']), None)
            if tool is None:
                messages.append({"role": "tool_outputs", "tool_outputs": None, "tool_error": f"No tool named <tool_name>{tool_input['tool_name']}</tool_name> available."})
                return {"next_action": "auto_respond", "messages": messages}

            tool_result = tool.use_tool(**tool_input['tool_arguments'])
            tool_outputs.append({"tool_name": tool_input['tool_name'], "tool_result": tool_result})
        
        messages.append({"role": "tool_outputs", "tool_outputs": tool_outputs, "tool_error": None})
        return {"next_action": "auto_respond", "messages": messages}
    else:
        raise ValueError(f"Provided role should be assistant or tool_inputs, got {claude_res['role']}")

In [38]:
# Call the tool_user with a prompt to get a version of Claude that can use your tools!
user_message = {"role": "user", "content": "Reverse the string 'encyclopeida'"}
messages = [user_message]

In [39]:
while True:
    claude_res = tool_user.use_tools(messages, execution_mode='manual')
    result = handle_manual_claude_res(messages, claude_res, tool_user)
    messages = result['messages']

    if result['next_action'] == 'user_input':
        print(claude_res['content'])
        break
    elif result['next_action'] == 'auto_respond':
        continue



The reversed string is 'adiepolcycne'.



In this updated code:

1. We define the `handle_manual_claude_res` function to process Claude's responses in manual mode. This function checks the role of Claude's response and determines the next action accordingly.

2. We start a loop that continues until Claude returns a response with the role 'assistant', indicating that it has completed the task.

3. Inside the loop, we call `tool_user.use_tools` with `execution_mode='manual'` to get Claude's response.

4. We pass Claude's response to the `handle_manual_claude_res` function, which processes the response and returns the next action and updated messages.

5. If the next action is 'user_input', we print Claude's response and break the loop, as the task is complete.

6. If the next action is 'auto_respond', we continue the loop to process the next response from Claude.
